In [ ]:
import optuna
from optuna.trial import TrialState

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data.dataloader import DataLoader

In [ ]:
#ToTensor: Grayscale image (RGB 0)~255 to 0~Normalize to the range of 1), Normalize: Z-value (RGB mean and standard deviation to 0).Normalize with 5)
transform = transforms.Compose([transforms.ToTensor()])#, transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

#Download training data
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)


#Download test data
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

In [ ]:
#Training dataset: 50,000 RGB images with 32 pixels in height and width
print(train_dataset.data.shape)
(50000, 32, 32, 3)

#Test dataset: 10000 RGB images with 32 pixels in height and width
print(test_dataset.data.shape)
(10000, 32, 32, 3)

#Check the class list
print(train_dataset.classes)
['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

#Classes are often used, so keep them separately
classes = train_dataset.classes

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))
        
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [ ]:
#Implement CNN
class Cifar10CnnModel(ImageClassificationBase):
    def __init__(self):
        
        super().__init__()
        self.network = nn.Sequential(
        
        nn.Conv2d(3, 16, 3,1,1),
        nn.ReLU(),    
        nn.Conv2d(16, 16, 3,1,1),
        nn.ReLU(),
        nn.MaxPool2d(2, 2),
        
        nn.Conv2d(16, 32, 3,1,1),
        nn.ReLU(),
        nn.Conv2d(32, 32, 3,1,1),
        nn.ReLU(),
        nn.MaxPool2d(2, 2),
            
        nn.Conv2d(32, 64, 3,1,1),
        nn.ReLU(),
        nn.Conv2d(64, 64, 3,1,1),
        nn.ReLU(),
        nn.MaxPool2d(2, 2),
            
        nn.Conv2d(64, 128, 3,1,1),
        nn.ReLU(),
        nn.Conv2d(128, 128, 3,1,1),
        nn.ReLU(),
        nn.AdaptiveAvgPool2d(1),
        
        nn.Flatten(),
        nn.Linear(128, 10))
        
    def forward(self, x):
        return self.network(x)

model = Cifar10CnnModel()
model


In [ ]:
random_seed = 42
torch.manual_seed(random_seed);

In [ ]:
from torch.utils.data import random_split

val_size = 5000
train_size = len(train_dataset) - val_size

train_ds, val_ds = random_split(train_dataset, [train_size, val_size])
len(train_ds), len(val_ds)


In [ ]:
batch_size = 128
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=4,pin_memory=True)
val_loader = DataLoader(val_ds, shuffle=False,num_workers=4,pin_memory=True)
test_loader = DataLoader(test_dataset, shuffle=False,num_workers=4,pin_memory=True)

In [ ]:
from torchvision.utils import make_grid

def show_batch(dl):
    for images, labels in dl:
        fig, ax = plt.subplots(figsize=(12, 6))
        ax.set_xticks([]); ax.set_yticks([])
        ax.imshow(make_grid(images, nrow=16).permute(1, 2, 0))
        break

In [ ]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [ ]:
device = get_default_device()
device

In [ ]:
show_batch(train_loader)

### Test output

In [ ]:
for images, labels in train_loader:
    print('images.shape:', images.shape)
    out = model(images)
    print('out.shape:', out.shape)
    print('out[0]:', out[0])
    break

In [ ]:
train_dl = DeviceDataLoader(train_loader,device)
val_dl = DeviceDataLoader(val_loader,device)
test_dl = DeviceDataLoader(test_loader,device)

# try different scheduler

In [ ]:
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

In [ ]:
model = to_device(Cifar10CnnModel(), device)

In [ ]:
result=evaluate(model, val_dl)

In [ ]:
result['val_acc']

In [ ]:
evaluate(model, test_dl)

In [ ]:
from torch.optim.lr_scheduler import ExponentialLR
from torch.optim.lr_scheduler import StepLR
from torch.optim.lr_scheduler import ReduceLROnPlateau
#Cross entropy

num_epochs = 50

lr = 0.000859941832304690
opt_func = torch.optim.Adam

model = to_device(Cifar10CnnModel(), device)

In [ ]:
batch_size = 62
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=4,pin_memory=True)
val_loader = DataLoader(val_ds, shuffle=False,num_workers=4,pin_memory=True)
test_loader = DataLoader(test_dataset, shuffle=False,num_workers=4,pin_memory=True)

train_dl = DeviceDataLoader(train_loader,device)
val_dl = DeviceDataLoader(val_loader,device)
test_dl = DeviceDataLoader(test_loader,device)

In [ ]:
def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    
    model = to_device(Cifar10CnnModel(), device)
    
    history = []
    lrs = []
    optimizer = opt_func(model.parameters(), lr,weight_decay=1e-4)
    #scheduler = ReduceLROnPlateau(optimizer, 'min')
    
    #scheduler_2 = ReduceLROnPlateau(optimizer)
    for epoch in range(epochs):     
        torch.manual_seed(np.random.randint(5000));
        train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=4,pin_memory=True)
        train_dl = DeviceDataLoader(train_loader,device)
        
        # Training Phase 
        model.train()
        train_losses = []
        for batch in train_dl:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()       
            optimizer.zero_grad()
        
        result = evaluate(model, val_dl)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        lrs.append(optimizer.param_groups[0]["lr"])
        
        #scheduler.step(result['val_loss'])
        #print(result['val_loss'])
        
        model.epoch_end(epoch, result)
        history.append(result)
        
    #torch.optim.swa_utils.update_bn(loader, swa_model)
    # Use swa_model to make predictions on test data
    #preds = swa_model(test_input)    
    return history,lrs

In [ ]:
history,lrs = fit(num_epochs, lr, model, train_ds, val_dl, opt_func)

In [ ]:
get_default_device()

# Try "Optuna"

In [ ]:
EPOCHS =20
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
def objective(trial):    
    # Generate the model.
    model = to_device(Cifar10CnnModel(), device)
        
    lr = trial.suggest_float("lr", 1e-6, 1e-2, log=True)
    batch_size = trial.suggest_int("batch_size",16,1024) 
    
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=4,pin_memory=True)
    valid_loader = DataLoader(val_ds, shuffle=False,num_workers=4,pin_memory=True)
    
    
    train_dl = DeviceDataLoader(train_loader,device)
    val_dl = DeviceDataLoader(valid_loader,device)
    
    
    optimizer = torch.optim.Adam(model.parameters(), lr)
    
 # Training of the model.
    for epoch in range(EPOCHS):
        
        torch.manual_seed(np.random.randint(5000));
        train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=4,pin_memory=True)
        train_dl = DeviceDataLoader(train_loader,device)
        
        model.train()
            
        for idx,batch in enumerate(train_dl):
            loss = model.training_step(batch)
            #train_losses.append(loss)
            loss.backward()
            optimizer.step()       
            optimizer.zero_grad()
            
            #if idx>30:
            #    break
            

        # Validation of the model.
        with torch.no_grad():
            model.eval()
            outputs = [model.validation_step(batch) for batch in val_dl]
            result=model.validation_epoch_end(outputs)
            
        print("epoch=%d, accuracy=%f"%(epoch,result['val_acc'])) 
        trial.report(result['val_acc'],epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
    return result['val_acc']

In [ ]:
if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=500, timeout=72000)

    pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))